In [1]:
!pip install tensorflow_addons

In [2]:
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
import os
import argparse
import matplotlib
# importing required libraries
import tensorflow as tf 
import tensorflow_addons as tfa
import numpy as np
import pandas as pd
import os
import cv2
from skimage import color
import scipy
import math

import sklearn
from sklearn.model_selection import train_test_split

import keras
from keras import callbacks
from keras import optimizers
#from keras.engine import Model
from keras.layers import Dropout, Flatten, Dense, Activation, BatchNormalization
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint

from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions

from google.colab import drive
from google.colab.patches import cv2_imshow
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
from Adam import Adam_Metaplastic
from data_utils import *
from BNN_Conv import *

In [4]:
def rescale(i, image, width, height):
  new_im = cv2.resize(image,(width,height))
  return new_im

In [5]:
# FOR INITIAL RAW IMAGES
faulty_dir = '/content/drive/Shareddrives/Senior Thesis/Casting_image/casting_512x512/casting_512x512/def_front/'
ok_dir = '/content/drive/Shareddrives/Senior Thesis/Casting_image/casting_512x512/casting_512x512/ok_front/'

# FOR AUGMENTED IMAGES
#faulty_dir = '/content/drive/Shareddrives/Senior Thesis/Casting_image/casting_data/casting_data/images/def_front/'
#ok_dir = '/content/drive/Shareddrives/Senior Thesis/Casting_image/casting_data/casting_data/images/ok_front/'
directories = (faulty_dir, ok_dir)

# check the total number of images we have
count = 0
for direct in directories:
  for file in os.listdir(direct):
    count += 1

# set the values of the X and y arrays
num_images = count
dim = 100
input_shape = (dim, dim, 3)
height = input_shape[0]
width = input_shape[1]
channels = input_shape[2]

X = np.zeros((num_images,height,width,channels))
y = np.zeros((num_images))

# populate the arrays
i = 0
j = -1
for direct in directories:
  j+=1
  for file in os.listdir(direct):
    #print(file)
    path = (direct+file)
    im = plt.imread(path)
    scaled_im = rescale(i, im, width, height)
    #edges, gradient = filter(scaled_im[:,:,0])
    X[i] = scaled_im
    if j == 0:
      # 1 for faulty
      y[i] = 1
    else:
      # 0 for ok image
      y[i] = 0
    i+=1

In [6]:
from sklearn.decomposition import PCA
dims = 50
X_train = X
y_train = y
X_train_fit = X_train.reshape(X_train.shape[0], X_train.shape[1]*X_train.shape[2]*X_train.shape[3])
pca = PCA(0.99)
X_train_fit = pca.fit_transform(X_train_fit)
#kmeans clustering on PCA data
from sklearn.cluster import AgglomerativeClustering
data = AgglomerativeClustering(n_clusters = 3, compute_distances=True).fit(X_train_fit)

In [7]:
# import tensorflow as tf
# MyModel = tf.keras.models.Sequential()
# MyModel.add(tf.keras.applications.ResNet50(include_top = False, weights='imagenet', pooling='avg',))
# MyModel.layers[0].trainable = False
# img = tf.keras.applications.resnet50.preprocess_input(X_train)
# extractedFeatures = MyModel.predict(img)
# pca_processed = PCA(0.99)
# X_processed = pca_processed.fit_transform(extractedFeatures)
# data_processed = AgglomerativeClustering(n_clusters = 3, compute_distances=True).fit(X_processed)

In [8]:
path_name = "/content/drive/Shareddrives/Senior Thesis/Models/my_model_1__forSegmentation_100.hdf5"
segModel = tf.keras.models.load_model(path_name)
last = segModel.layers[-2].output
fullmodel = tf.keras.Model(segModel.input, last)
fullmodel.summary()
extractedFeatures = fullmodel.predict(X_train)
pca_processed = PCA(n_components = 50)
X_processed = pca_processed.fit_transform(extractedFeatures)
data_processed = AgglomerativeClustering(n_clusters = 3, compute_distances=True).fit(X_processed)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100, 100, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 100, 100, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 100, 100, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 50, 50, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 50, 50, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 50, 50, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 25, 25, 128)       0     

In [9]:
from mpl_toolkits import mplot3d
%matplotlib inline
import matplotlib

In [10]:
idx_train0 = np.array(np.where(y_train==0)).flatten()
idx_train1 = np.array(np.where(y_train==1)).flatten()
idx_labels0 = np.array(np.where(data_processed.labels_==0)).flatten()
idx_labels1 = np.array(np.where(data_processed.labels_==1)).flatten()
idx_labels2 = np.array(np.where(data_processed.labels_==2)).flatten()

mapping = np.zeros((2,3))

mapping[0,0] = len(np.intersect1d(idx_train0, idx_labels0))
mapping[0,1] = len(np.intersect1d(idx_train0, idx_labels1))
mapping[0,2] = len(np.intersect1d(idx_train0, idx_labels2))

mapping[1,0] = len(np.intersect1d(idx_train1, idx_labels0))
mapping[1,1] = len(np.intersect1d(idx_train1, idx_labels1))
mapping[1,2] = len(np.intersect1d(idx_train1, idx_labels2))

print(mapping)

[[154. 280.  85.]
 [431. 146. 204.]]


In [11]:
y[np.intersect1d(idx_train1, idx_labels0)] = 1 #defective type 1
y[np.intersect1d(idx_train1, idx_labels2)] = 2 #defective type 2
print(np.array(np.where(y==0)).flatten().shape)
print(np.array(np.where(y==1)).flatten().shape)
print(np.array(np.where(y==2)).flatten().shape)

(519,)
(577,)
(204,)


In [12]:
X_normal = X[np.where(y==0)]
y_normal = y[np.where(y==0)]
y_def1 = y[np.where(y==1)]
X_def1 = X[np.where(y==1)]
y_def2 = y[np.where(y==2)]
X_def2 = X[np.where(y==2)]

In [13]:
norm_data = len(X_normal)
def1_data = len(X_def1)
def2_data = len(X_def2)

In [14]:
X_task1 = np.concatenate((X_normal, X_def1[0:norm_data]))
y_task1 = np.concatenate((y_normal, y_def1[0:norm_data]))
X_task2 = np.concatenate((X_normal[0:def2_data], X_def2))
y_task2 = np.concatenate((y_normal[0:def2_data], y_def2))
y_task2[np.where(y_task2==2)] = 1

print(X_task1.shape)
print(y_task1.shape)
print(X_task2.shape)
print(y_task2.shape)

(1038, 100, 100, 3)
(1038,)
(408, 100, 100, 3)
(408,)


In [15]:
from keras.utils.np_utils import to_categorical
y_task1_cat = to_categorical(y_task1)
y_task2_cat = to_categorical(y_task2)
print(y_task1_cat.shape)
print(y_task2_cat.shape)

(1038, 2)
(408, 2)


In [16]:
from sklearn.model_selection import train_test_split
frac = 0.3

X_task1_train, X_task1_test, y_task1_train, y_task1_test = train_test_split(X_task1, y_task1_cat, test_size = frac, random_state=1)
X_task2_train, X_task2_test, y_task2_train, y_task2_test  = train_test_split(X_task2, y_task2_cat, test_size = frac, random_state=1)
print(X_task1_train.shape)
print(X_task2_train.shape)

(726, 100, 100, 3)
(285, 100, 100, 3)


In [17]:
class CustomImageDataset(torch.utils.data.Dataset):
  def __init__(self, images, labels):
          self.labels = labels
          self.images = images
  def __len__(self):
          return len(self.labels)
  def __getitem__(self, idx):
          label = self.labels[idx]
          image = self.images[idx]
          sample = {'image': image, 'label': label}
          return sample

In [18]:
X_train = torch.Tensor(np.transpose(X_task1_train,(0,3,1,2)))
X_test = torch.Tensor(np.transpose(X_task1_test,(0,3,1,2)))
print(X_train.shape)
print(X_test.shape)
train_data = CustomImageDataset(X_train,y_task1_train)
test_data = CustomImageDataset(X_test,y_task1_test)
train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=1, shuffle=True)
test_data_loader = torch.utils.data.DataLoader(test_data, batch_size=1, shuffle=True)

torch.Size([726, 3, 100, 100])
torch.Size([312, 3, 100, 100])


**Normal Model**

In [19]:
import torch.nn as nn
class CNN(nn.Module):
    def __init__(self, width = 0.01, norm = 'batch'):
        super(CNN, self).__init__()

        self.hidden_layers = 2

        # modeled from example_model_initialization in COS429 Student Assignment 3 - Master
        # define network topology
        conv1 = nn.Conv2d(3, 32, kernel_size=5, padding = 2, stride = 2, bias=False)
        norm1 = self.normalization(32,2,norm)
        conv2 = nn.Conv2d(32, 64, kernel_size=4, padding = 2, stride =2, bias=False)
        norm2 = self.normalization(64,2,norm)
        linear = nn.Linear(43264,3, bias=False)
        norm3 = self.normalization(3,1,norm)
        act3 = nn.Softmax(dim=0)

        # pytorch ModuleDict https://pytorch.org/docs/stable/generated/torch.nn.ModuleDict.html
        self.layers = nn.ModuleDict({'cv1': conv1, 'n1': norm1, 'cv2': conv2, 'n2': norm2, 'fc':linear, 'n3':norm3, 'a3':act3})

        # initialize weights
        # from pytorch init documentation https://pytorch.org/docs/stable/nn.init.html
        nn.init.normal_(conv1.weight, mean=0, std=width)
        nn.init.normal_(conv2.weight, mean=0, std=width)
        nn.init.normal_(linear.weight, mean=0, std=width)
          
    def normalization(self, size, dim, norm):
        # from pytorch documentation: https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm2d.html
        if norm == 'batch':
            if dim==2:
                return nn.BatchNorm2d(size, affine=True, track_running_stats=True)
            else:
                return nn.BatchNorm1d(size, affine=True, track_running_stats=True)
        else:
            if dim==2:
                return nn.InstanceNorm2d(size, affine=False, track_running_stats=False)
            else:
                return nn.InstanceNorm1d(size, affine=False, track_running_stats=False)

    def forward(self, x):
        x = self.layers['cv1'](x)
        x = self.layers['n1'](x)

        x = self.layers['cv2'](x)
        x = self.layers['n2'](x)

        # flatten x before the fully connected layer via pytorch's view function
        x = x.view(x.size(0), -1)
        x = self.layers['fc'](x)
        x = self.layers['n3'](x)
        x = self.layers['a3'](x)

        return x
        

### Hyperparameters

In [26]:
# Hyperparameters
lr = 0.005
epochs = 10
meta = 1.35

#init = "normal"
init_width = 0.1
decay = 0
gamma = 1
norm = 'batch'
lrs = [lr*(gamma**(-i)) for i in range(epochs)]

## Define Model

In [21]:
model = ConvNet(width = init_width)

## Train and Test Functions


In [22]:
def train(model, train_loader, optimizer, batch_size, criterion = torch.nn.CrossEntropyLoss()):
    
    model.train()

    target = np.zeros((batch_size,2))
    data = np.zeros((batch_size,3,100,100))

    for i in range(batch_size):
      target[i,:]  = next(iter(train_loader))['label']
      data[i,:,:,:] = next(iter(train_loader))['image']

    target = torch.Tensor(target)
    data = torch.Tensor(data)

    optimizer.zero_grad()
    output = model.forward(data)
    loss = criterion(output, target)
    loss.backward() 
    optimizer.step()

In [23]:
def test(model, test_loader, batch_size, criterion = torch.nn.CrossEntropyLoss(reduction='sum'), verbose = False):
    
    model.eval()
    test_loss = 0
    correct = 0
    
    target = np.zeros((batch_size,2))
    data = np.zeros((batch_size,3,100,100))

    for i in range(batch_size):
      target[i,:]  = next(iter(test_loader))['label']
      data[i,:,:,:] = next(iter(test_loader))['image']
    
    target = torch.Tensor(target)
    data = torch.Tensor(data)

    output = model.forward(data)
    test_loss = criterion(output, target).item() # mean batch loss
    pred = output.max(1, keepdim=True)[1].flatten() # get the index of the max log-probability
    target_label = target.max(1, keepdim=True)[1].flatten()
    correct = torch.eq(target_label, pred).sum()

    test_loss /= batch_size
    test_acc = 100 * correct / batch_size
    
    print('Test accuracy: ', (correct, test_acc))

    return test_acc, test_loss

## Run Model

In [27]:
optimizer = Adam_Metaplastic(model.parameters(), lr = 0.005, meta = meta, weight_decay = decay)
        
for epoch in range(1, epochs+1):
  train(model, train_data_loader, optimizer, batch_size = 128)

  print(f"EPOCH: {epoch}")
  test_accuracy, test_loss = test(model, test_data_loader, batch_size = 64)
  
  #current_bn_state = model.save_bn_states()
  #test_accuracy, test_loss = test(model, other_task, verbose=True)

/content/BNN_Conv.py:125: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.layers['a3'](x)


EPOCH: 1
Test accuracy:  (tensor(25), tensor(39.0625))
EPOCH: 2
Test accuracy:  (tensor(32), tensor(50.))
EPOCH: 3
Test accuracy:  (tensor(34), tensor(53.1250))
EPOCH: 4
Test accuracy:  (tensor(30), tensor(46.8750))
EPOCH: 5
Test accuracy:  (tensor(36), tensor(56.2500))
EPOCH: 6
Test accuracy:  (tensor(28), tensor(43.7500))
EPOCH: 7
Test accuracy:  (tensor(33), tensor(51.5625))
EPOCH: 8
Test accuracy:  (tensor(36), tensor(56.2500))
EPOCH: 9
Test accuracy:  (tensor(32), tensor(50.))
EPOCH: 10
Test accuracy:  (tensor(36), tensor(56.2500))


In [ ]:
from keras.applications.vgg16 import VGG16
base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
# We freeze every layer in our base model so that they do not train, we want that our feature extractor stays as before --> transfer learning
for layer in base_model.layers: 
  layer.trainable = False
  print('Layer ' + layer.name + ' frozen.')
# We take the last layer of our the model and add it to our classifier
last = base_model.layers[-1].output
x = Flatten()(last)
x = Dense(1000, activation='relu', name='fc1')(x)
x = Dropout(0.5)(x)
x = Dense(2, activation='softmax', name='predictions')(x)
model = tf.keras.Model(base_model.input, x)

# We compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
loss_entropy = tf.keras.losses.CategoricalCrossentropy()
loss_binary_entropy = tf.keras.losses.BinaryCrossentropy()
model.compile(optimizer=optimizer, loss=loss_entropy, metrics=['accuracy'])
from keras.utils.vis_utils import plot_model
#plot_model(model, to_file='classifier_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
# We start the training
#tf.config.experimental_run_functions_eagerly(True)
config = 1
transform = "2class_sequential"
preprocess = ""
imsize = str(dim)
path_name = f"{'/content/drive/Shareddrives/Senior Thesis/Models/my_model_'}{config}{'_'}{preprocess}{'_'}{transform}{'_'}{imsize}{'.hdf5'}"
earlyStopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save = tf.keras.callbacks.ModelCheckpoint(path_name, save_best_only=True, monitor='val_loss', mode='min')
reduce_lr_loss = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=7, verbose=1, min_delta=1e-4, mode='min')

In [ ]:
val_accuracy = []
other_val_accuracy = []
for i in range(20):
  model.fit(x=X_task2_train, y=y_task2_train, batch_size=64, epochs=1, verbose=1, callbacks=[mcp_save, reduce_lr_loss], validation_data=(X_task2_test,y_task2_test), shuffle=True)
  other_val_accuracy.append(model.history.history['val_accuracy'])
  loss, acc  = model.evaluate(X_task1_test, y_task1_test)
  val_accuracy.append(acc)
for i in range(20):
  model.fit(x=X_task1_train, y=y_task1_train, batch_size=64, epochs=1, verbose=1, callbacks=[mcp_save, reduce_lr_loss], validation_data=(X_task1_test,y_task1_test), shuffle=True)
  val_accuracy.append(model.history.history['val_accuracy'])
  loss, acc  = model.evaluate(X_task2_test, y_task2_test)
  other_val_accuracy.append(acc)

In [ ]:
task1 = np.zeros((40))
task2 = np.zeros((40))
for i in range(len(val_accuracy)):
  if type(val_accuracy[i]) == list:
    task1[i] = val_accuracy[i][0]
  else:
    task1[i] = val_accuracy[i]

for i in range(len(other_val_accuracy)):
  if type(other_val_accuracy[i]) == list:
    task2[i] = other_val_accuracy[i][0]
  else:
    task2[i] = other_val_accuracy[i]

In [ ]:
# accuracy = model.history.history['accuracy']
# val_accuracy = model.history.history['val_accuracy']
# loss = model.history.history['loss']
# val_loss = model.history.history['val_loss']

In [ ]:
import pandas as pd

data = np.zeros((40,2))
data[:,0] = task1
data[:,1] = task2
#data[:N,2] = loss
#data[:N,3] = val_loss

df = pd.DataFrame(data)
df.to_excel(excel_writer = "/content/data.xlsx")

In [ ]:
import time
start = time.time()
model.predict(X)
print((time.time()-start)/len(X))